# APIs para modelos de Machine Learning

## Introducción

En este notebook vamos a ver cómo podemos exponer un modelo de Machine Learning a través de una API. Para ello, vamos a utilizar la librería `fastapi` que nos permite crear APIs de forma sencilla y rápida. También trabajaremos con scikit-learn para entrenar un modelo de clasificación y guardarlo en disco para luego exponerlo a través de la API.

Utilizaremos el dataset de UCI llamado `adult` que contiene información sobre personas y su salario. El objetivo es predecir si una persona gana más de 50k al año o no.

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  
# metadata 
print(adult.metadata) 
  
# variable information 
print(adult.variables) 

{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Mon Aug 07 2023', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': 'Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the following conditions: ((AAG

En este ejercicio consideraremos tres componentes importantes en la creación de un modelo de una clasificación supervisada. Decimos que es supervisada porque el modelo se entrena con ejemplos que ya tienen la etiqueta que queremos predecir.

1. **Preprocesamiento de datos**: En esta etapa vamos a cargar el dataset, limpiarlo y transformar las variables categóricas en numéricas.

2. **Entrenamiento del modelo**: Utilizaremos un modelo de clasificación llamado `RandomForestClassifier` que se encuentra en la librería `scikit-learn`. Este modelo es un ensamble de árboles de decisión.

3. **Predicción**: Finalmente, vamos a exponer el modelo a través de una API para que cualquier persona pueda hacer predicciones.


Compilaremos estas tres componentes a traves de un Pipeline de scikit-learn, es muy importante tener en cuenta que este proceso aplica para cualquier modelo de clasificación supervisada, LO QUE IMPLICA QUE PUEDE REUTILIZARSE Y REVISARSE CUANDO ESTUDIE OTROS MODELOS DE MACHINE LEARNING.

## Como funciona un modelo de clasificación supervisada

Un modelo de clasificación supervisada es un algoritmo que aprende a predecir la etiqueta de una variable objetivo a partir de un conjunto de variables predictoras. En nuestro caso, queremos predecir si una persona gana más de 50k al año o no. Por lo tanto, la variable objetivo es `income` y las variables predictoras son `age`, `workclass`, `education`, `marital-status`, `occupation`, `relationship`, `race`, `sex`, `capital-gain`, `capital-loss`, `hours-per-week` y `native-country`.

La idea es que al exponer datos que el modelo no ha visto antes, pueda predecir la etiqueta de la variable objetivo. Por eso decimos que el modelo aprendió a partir de ejemplos previos.

### Exploracion de datos

Vamos a cargar el dataset y ver algunas estadísticas descriptivas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Transformación de los datos en un dataframe de pandas

df = pd.DataFrame(X, columns=adult.variables.name[:-1])
df['Income'] = y

# Visualización de los primeros registros
df.head()

name,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
### Análisis exploratorio de datos

# Información general del dataframe

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  Income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [ ]:
### Análisis de valores nulos

# Verificación de valores nulos

df.isnull().sum()

name
age                 0
workclass         963
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
Income              0
dtype: int64

In [ ]:
### Análisis de valores duplicados

# Verificación de valores duplicados

df.duplicated().sum()

29

In [ ]:
### Análisis de la variable objetivo

# Distribución de la variable objetivo

df['Income'].value_counts()



Income
<=50K     24720
<=50K.    12435
>50K       7841
>50K.      3846
Name: count, dtype: int64

In [ ]:
### Arreglo de la variable objetivo

# Reemplazo de los valores de la variable objetivo

df['Income'] = df['Income'].replace({'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1})
df['Income'].value_counts()

Income
0    37155
1    11687
Name: count, dtype: int64

In [ ]:
### Análisis de las variables numéricas

# Estadísticas descriptivas de las variables numéricas

df.describe()

name,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,Income
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382,0.239282
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444,0.426649
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000


In [ ]:
### Análisis de las variables categóricas

# Estadísticas descriptivas de las variables categóricas

df.describe(include='object')

name,workclass,education,marital-status,occupation,relationship,race,sex,native-country
count,47879,48842,48842,47876,48842,48842,48842,48568
unique,9,16,7,15,6,5,2,42
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
freq,33906,15784,22379,6172,19716,41762,32650,43832


In [ ]:
### Visualización de la variable objetivo

# Gráfico de barras de la variable objetivo

plt.figure(figsize=(8, 6))
plt.title('Distribución de la variable objetivo')
plt.xlabel('Ingresos')
plt.ylabel('Cantidad de registros')
df['Income'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
### tratamiento de variables 

# Transformación de las variables categóricas

df['workclass'].value_counts()

In [ ]:
####